In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%config Completer.use_jedi = False

In [3]:
import gc
import matplotlib.pyplot as plt
import pickle
import torch
import tqdm
import warnings
from optuna.exceptions import ExperimentalWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ExperimentalWarning)

In [10]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import array_contains, col, explode, split
from pyspark.sql import functions as sf

from replay.data_preparator import DataPreparator
from replay.experiment import Experiment
from replay.models.neuromf import NeuroMF
from replay.models.rl.ddpg_current_recommender import DDPG, ActorDRR, Env, StateReprModule, OUNoise
from replay.metrics import Coverage, HitRate, MRR, MAP, NDCG, Surprisal
from replay.session_handler import State
from replay.splitters import UserSplitter
from replay.utils import convert2spark

In [11]:
spark = State().session
spark

In [12]:
K=10
SEED=12345
K_list_metrics = [1, 5, 10]

## Data reading

In [18]:
data_path = '/Users/kuderov/data/sberzvuk/ailab_data/'
data = spark.read.parquet(data_path + 'test_log.parquet')
data = test_log.withColumnRenamed('session_id', 'user_id').drop('relevance').groupBy("user_id", "item_id").agg(sf.max("timestamp").alias("timestamp"))
data = test_log.orderBy(["user_id", "timestamp"]).withColumn('relevance', sf.lit(1))
data = test_log.withColumnRenamed('user_id', 'user_idx').withColumnRenamed('item_id', 'item_idx')
data = test_log.filter(sf.col('user_idx') < 400000)
data.count()

AnalysisException: Path does not exist: file:/Users/kuderov/data/sberzvuk/ailab_data/test_log.parquet